# 02 — Test Map Generation

Generate a map for a single grid cell to test styling, resolution, and layout before running the full batch.

**What this notebook covers:**
- Loading config and boundary data
- Generating a single test map with `MapGenerator`
- Tweaking map styling (colors, fonts, basemap)
- Viewing the output PNG inline

## Setup

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.config_loader import load_config, get_data_dir, get_output_dir
from src.data_processing.load_boundaries import (
    load_control_grid,
    load_layer,
    validate_crs,
)
from src.mapping.map_generator import MapGenerator

config = load_config()
data_dir = get_data_dir(config)
output_dir = get_output_dir(config)

print(f"Data dir:   {data_dir}")
print(f"Output dir: {output_dir}")

## Load Data

In [ ]:
# Load control grid
grid = load_control_grid(data_dir)
grid = validate_crs(grid)
print(f"Grid cells: {len(grid)}")
print(f"Columns:    {list(grid.columns)}")

# Load optional context layers
roads = load_layer(data_dir, "roads")
buildings = load_layer(data_dir, "buildings")

## Pick a Test Grid Cell

Select the first grid cell for a quick test. Change the index or ID below to test a different cell.

In [ ]:
# Pick the first grid cell for testing
test_idx = 0
test_cell = grid.iloc[[test_idx]]

# Determine grid ID — adjust the column name to match your data
# Common candidates: CELL_ID, cell_id, grid_id, ID, NAME
grid_id_col = None  # Set to your column name, e.g. "CELL_ID"

if grid_id_col and grid_id_col in grid.columns:
    test_grid_id = str(test_cell[grid_id_col].iloc[0])
else:
    test_grid_id = str(test_idx + 1).zfill(3)

print(f"Test grid cell ID: {test_grid_id}")
test_cell

## Generate a Single Map

Create a `MapGenerator` and render one map. The output will be saved as a PNG file.

In [ ]:
# Create generator — save test outputs to a local test_output folder
test_output = PROJECT_ROOT / "test_output"

generator = MapGenerator(
    output_dir=test_output,
    fig_width=19.2,
    fig_height=10.8,
    dpi=100,
    add_basemap=True,  # Set to False if working offline
)

output_path = generator.generate_map(
    grid_cell=test_cell,
    grid_id=test_grid_id,
    all_grid_cells=grid,
    roads=roads,
    buildings=buildings,
)

print(f"Map saved to: {output_path}")

## View the Generated Map

Display the PNG inline to check quality and layout.

In [ ]:
from IPython.display import Image, display

display(Image(filename=str(output_path), width=960))

## Experiment with Styling

Try adjusting map parameters below to find the best look for tablet display.

In [ ]:
# Example: Higher DPI, different buffer
generator_hd = MapGenerator(
    output_dir=test_output / "hd_test",
    fig_width=19.2,
    fig_height=10.8,
    dpi=150,           # Higher resolution
    add_basemap=True,
)

output_hd = generator_hd.generate_map(
    grid_cell=test_cell,
    grid_id=test_grid_id,
    all_grid_cells=grid,
    roads=roads,
    buildings=buildings,
    buffer_factor=0.5,  # Wider view around the cell
)

print(f"HD map saved to: {output_hd}")
display(Image(filename=str(output_hd), width=960))

## Next Steps

Once you're happy with the styling:
- Open **03_batch_processing.ipynb** to generate maps for all grid cells
- Or run `python scripts/generate_all_maps.py` from the command line